In [1]:
from resume_parser import resumeparse
from tika import parser  
imwhole_projectport locationtagger
import re
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
from pdfminer.high_level import extract_text
import aspose.words as aw
import pandas as pd
import docx2txt
from PyPDF2 import PdfReader

####################################################
import os
import pathlib
import glob
from pdfminer.high_level import extract_text
import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_lg")
import docx


/home/anush/anaconda3/lib/python3.9/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [2]:
base_dir = pathlib.Path(__name__).parent.absolute()
pdf_folder_path = os.path.join(base_dir,'Training_resume',)
pdf_file_path  = os.path.join(base_dir,'Training_resume','*')
pdf_file_path

'/home/anush/Desktop/flask_name/Training_resume/*'

In [3]:
def doc_convert_to_pdf(path):
#     count =0
    cunt =0
    cnt = 0
    for file_ in glob.glob(path):
        try:
            if file_.endswith(".doc"):
                print(file_,'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa')
                doc = aw.Document(file_)
                doc.save(os.path.join(base_dir,'Training_resume',f"+{cunt}.pdf"))
                cunt+=1
            elif file_.endswith(".rtf"):
                print(file_,'bbbbbbbbbbbbbbbbbbbbbbbbbbbbbb')
                doc = aw.Document(file_)
                doc.save(os.path.join(base_dir,'Training_resume',f"++{cunt}.pdf"))
                cnt+=1        
        except  Exception as e:
            print(e)


#convert word to text
def  extract_text_from_word(word_path):
    doc = docx.Document(word_path)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)


#convert pdf to text
def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)


#return text
def return_text(path):
    if path.endswith('.docx'):
        return extract_text_from_word(path)
    elif path.endswith('.pdf'):
        return extract_text_from_pdf(path)
    

#convert_the_info_dic
def convert_the_info_dic(text):
    #breaking the text
    dit ={}
    title = ['Education','Qualification','Skills','Objective','Experience','Tools','Designation','Employment']
    for i in title:
        text = text.replace(f'{i}',f'|{i}').replace(f'{i.upper()}',f'|{i.upper()}')
    text = text.split("|")

    #making dictionary of inforamtion
    for titl in title:
        check = False
        for txt in text: 
            if titl in txt or titl.upper() in txt:
                check = True
                dit[titl] = txt 
        if check == False:
            dit[titl] = 'Nan'
    return dit


#extraction_the_data
def extraction_of_Data(resume_path,data_in_dic,text):
    prev =[]
    person =[]
    city = []
    education = []
    skill =[]
    doc = nlp(text)


#     extraction of Education
    edu_attribute = data_in_dic['Education']
    education.append(edu_attribute)
    
    data1 = resumeparse.read_file(resume_path)
    #extract the name
    name_data = data1['name']
    if 'Aspose Pty' == name_data:
        people =[]
        for ent in doc.ents:
            if ent.label_ == "PERSON" or (ent.text.istitle()):
                people.append(ent.text)
        name_data = people[2]
    elif 'curriculum vitae' == name_data.lower():
        people =[]
        for ent in doc.ents:
            if ent.label_ == "PERSON" or (ent.text.istitle()):
                people.append(ent.text)
                break
        name_data = people[0]
    else:
        name_data = name_data
    #extract the email
    email_data = data1['email']
    #extract the phone
    phone = data1['phone']
    if str(phone)=='003-2023':
        phone_pattern = re.compile(r'(?:\+\d{1,2}\s)?\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}')
        phone_numbers = re.findall(phone_pattern, text)
        phone =phone_numbers[0]
    else:
        phone = phone

    #extract the designition
    designation_data = data1['designition']
    
    #extract the experience
    experience_data = data1['total_exp']
    if len([i for i in str(experience_data)])>2:
        experience_data = None
    else:
        experience_data = experience_data
        
    #extract the skills
#     skill_data = data1['skills']
        
    #extract the New_skills    
    skill_attribute = data_in_dic['Skills']
    if str(skill_attribute) =='Nan':
        skill.append(data1['skills'])
    else:
        skill_attribute = data_in_dic['Skills']
        skill.append(skill_attribute)
    
    #append the information 
#     prev.append(person)
    prev.append(name_data)
    prev.append(email_data)
    prev.append(phone)
    prev.append(skill)
    prev.append(designation_data)
    prev.append(experience_data)
    prev.append(education)
#     prev.append(city)
#location
    place_entity = locationtagger.find_locations(text = text)
    prev.append(str(place_entity.countries))
    prev.append(str(place_entity.regions))
    prev.append(str(place_entity.cities))
    prev.append(resume_path)
    return prev 

In [4]:
df = pd.DataFrame(columns=['Name', 'Email','Mobile','Skills','Designation','Experience_Period','education','countries','regions','cities','Resume_path'])

In [5]:
def run_file(path_of_list):
    try:
        text = return_text(path_of_list)
        data_in_dic = convert_the_info_dic(text)
        data = extraction_of_Data(path_of_list,data_in_dic, text)
########################creating the dataframe###############################
        df.loc[-1] = data
        df.reset_index(drop ='index',inplace =True)
    except Exception as e:
        print(e)

doc_convert_to_pdf(pdf_file_path)

for path_of_list in glob.glob(pdf_file_path):
    if path_of_list.endswith(".pdf"):
        run_file(path_of_list)
    
    elif path_of_list.endswith(".docx"):
        run_file(path_of_list)

df  

/home/anush/Desktop/flask_name/Training_resume/RESUME.doc aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
/home/anush/Desktop/flask_name/Training_resume/Cecilia Morales Resume.rtf bbbbbbbbbbbbbbbbbbbbbbbbbbbbbb
/home/anush/Desktop/flask_name/Training_resume/Resume3.rtf bbbbbbbbbbbbbbbbbbbbbbbbbbbbbb
/home/anush/Desktop/flask_name/Training_resume/Document.rtf bbbbbbbbbbbbbbbbbbbbbbbbbbbbbb
/home/anush/Desktop/flask_name/Training_resume/Resume 2019.doc aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
/home/anush/Desktop/flask_name/Training_resume/JLittle2018Resume1c.doc aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa


KeyboardInterrupt: 

In [ ]:
def cities(x):
    x =x[1:-1]
    doc = nlp(x)
    citi=[]
    for ent in doc.ents:
        if ent.label_ == 'GPE'or ent.label_ =='NORP' or ent.label_ == 'FAC':
            citi.append(ent.text)
    return citi

df['cities'] = df['cities'].apply(lambda x :cities(x))

In [ ]:
df

# data_read

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# Load the resume and job data

resume_df =  pd.read_csv(os.path.join(base_dir,'data_csv','client22.csv'))
jobs_df = pd.read_csv(os.path.join(base_dir,'data_csv','naukari.csv'))

In [ ]:
resume_df.drop('Unnamed: 0',axis=1,inplace =True)

In [ ]:
data = resume_df.copy()

In [ ]:
#Name______________________________________________________
resume_df["Name"]=resume_df['Name'].apply(lambda x: str(x).lower().replace('CURRICULUM VITAE'.lower(),'Nan'))

In [ ]:
def data_list(x):
    try:
        return   'Nan' if x  == '[]' else ','.join(x[1:-1].split(','))
    except:
        return 'Nan'
resume_df['Designation']= resume_df['Designation'].apply(lambda x:data_list(x))
resume_df['Skills']= resume_df['Skills'].apply(lambda x:data_list(x))
resume_df['education']= resume_df['education'].apply(lambda x:data_list(x))
resume_df['cities']= resume_df['cities'].apply(lambda x:data_list(x))
resume_df['countries'] = resume_df['countries'].apply(lambda x:data_list(x))
resume_df['regions'] = resume_df['regions'].apply(lambda x:data_list(x))

In [ ]:
##education_____________
import re
resume_df['education'].replace(to_replace=[r"\\t|\\n|\\r", r'\r+|\n+|\t+',r"\t|\n|\r"], value=["","",""], regex=True, inplace=True)
ad = r'Created\s+with\s+an\s+evaluation\s+copy\s+of\s+Aspose\.Words\.\s+To\s+discover\s+the\s+full\s+versions\s+of\s+our\s+APIs\s+please\s+visit:\s+https://products\.aspose\.com/words/'
resume_df['education'] =resume_df['education'].apply(lambda x : re.sub(ad, "", x))
a,b,c,d,e= '\\x0c','\\xa0','\\u200b','Education','EDUCATION'
dat = [i.replace(a, '').replace(b,'').replace(c,'').replace(d,'').replace(e,'') for i in resume_df['education'].values]
resume_df['education'] = dat

In [ ]:
##email_________________________

dat = [ str(x).lower().replace('e-mail:-','').replace('e-mail :','').replace('email :-','').replace('email:','').replace('email :','').replace('e_mail :-','').replace('e_mail :','').replace('id:','').replace('mail :-','') for x in  resume_df['Email'].values]
resume_df['Email'] =dat

In [ ]:
#remove_Punctuation___________

import string
string.punctuation
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

resume_df['Skills']= resume_df['Skills'].apply(lambda x: remove_punct(x))
resume_df['Designation']= resume_df['Designation'].apply(lambda x: remove_punct(x))
resume_df['education']= resume_df['education'].apply(lambda x: remove_punct(x))
resume_df['countries']= resume_df['countries'].apply(lambda x: remove_punct(x))
resume_df['regions']= resume_df['regions'].apply(lambda x: remove_punct(x))
resume_df['cities']= resume_df['cities'].apply(lambda x: remove_punct(x))

resume_df.head(10)

In [ ]:
s = resume_df.iloc[0].values[3]

In [ ]:
z = ' '.join(s).replace("SKILLS ","")

In [ ]:
"Skills: nnSoftware Microsoft Powerpoint Outlook Excel Microsoft Word Typing wpmunOperate Avaya phone system nnAWARDSHONORS AND CERTIFICATIONS nnHonor Roll Student nnTeacher certification nnxc ,job description: required python engineer for backend development,Query:How much percentage is Skills is suitable for given job description,Percentage:"

In [ ]:
f

In [ ]:
def tokenization(text):
    text = re.split('\W+', str(text))
    return text

resume_df['Skills']= resume_df['Skills'].apply(lambda x: tokenization(x))
resume_df['Designation']= resume_df['Designation'].apply(lambda x: tokenization(x))
resume_df['education']= resume_df['education'].apply(lambda x: tokenization(x))
resume_df['countries']= resume_df['countries'].apply(lambda x: tokenization(x))
resume_df['regions']= resume_df['regions'].apply(lambda x: tokenization(x))
resume_df['cities']= resume_df['cities'].apply(lambda x: tokenization(x))

In [ ]:
# In this project firstly we extract the data from the resume.
# 4
# Then we recommended using content based filtering recommnedation engine
# 5
# For that we used cosine similarity for recommendation system.
# 6
import nltk
nltk.download('stopwords')
stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text


resume_df['Skills']= resume_df['Skills'].apply(lambda x: remove_stopwords(x))
resume_df['Designation']= resume_df['Designation'].apply(lambda x: remove_stopwords(x))
resume_df['education']= resume_df['education'].apply(lambda x: remove_stopwords(x))
resume_df['countries']= resume_df['countries'].apply(lambda x: remove_stopwords(x))
resume_df['regions']= resume_df['regions'].apply(lambda x: remove_stopwords(x))
resume_df['cities']= resume_df['cities'].apply(lambda x: remove_stopwords(x))

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
wn = nltk.WordNetLemmatizer()


wn = nltk.WordNetLemmatizer()
def get_wordnet_pos(treebank_tag):
    treebank_tag =str(treebank_tag)
    if treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
def lemmatzer(text):
    words_and_tags = nltk.pos_tag(text)
    lem = []
    for word, tag in words_and_tags:
        lemma = wn.lemmatize(word,pos =get_wordnet_pos(tag))
        lem.append(lemma)
    return lem

# resume_df['New_Skill']= resume_df['New_Skill'].apply(lambda x: lemmatzer(x))
resume_df['Skills']= resume_df['Skills'].apply(lambda x: lemmatzer(x))
resume_df['Designation']= resume_df['Designation'].apply(lambda x: lemmatzer(x))
resume_df['education']= resume_df['education'].apply(lambda x: lemmatzer(x))
resume_df['countries']= resume_df['countries'].apply(lambda x: lemmatzer(x))
resume_df['regions']= resume_df['regions'].apply(lambda x: lemmatzer(x))
resume_df['cities']= resume_df['cities'].apply(lambda x: lemmatzer(x))

In [ ]:
' '.join(resume_df['Skills'][0])

In [ ]:
dat = pd.DataFrame(columns =['Skills','Designation','education','countries','regions','cities'])

In [ ]:
def convert_into_string(x):
    return ''.join(x)
dat['Skills']= resume_df['Skills'].apply(lambda x: ' '.join(x))
dat['Designation']= resume_df['Designation'].apply(lambda x: ' '.join(x))
dat['education']= resume_df['education'].apply(lambda x: ' '.join(x))
dat['countries']= resume_df['countries'].apply(lambda x: ' '.join(x))
dat['regions']= resume_df['regions'].apply(lambda x: ' '.join(x))
dat['cities']= resume_df['cities'].apply(lambda x: ' '.join(x))

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
lst = ['Skills','Designation','education','countries','regions','cities']

def word_cloud(dat,lst):
    plt.figure(figsize=(6,6))
    count =1
    for col in lst:
        plt.subplot(3,2, count)
        wc = WordCloud(background_color="black", stopwords=STOPWORDS,width=250, height=180)
        wc.generate(" ".join(dat[str(col)]))
        plt.imshow(wc.recolor( colormap= 'Pastel2' , random_state=17), alpha=0.98)
        plt.title(str(col), fontsize=10)
        plt.axis('off')
        count+=1

word_cloud(dat,lst)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [ ]:
resume_df['Experience_Period']= resume_df['Experience_Period'].apply(lambda x: str(x)+' years')

In [ ]:
cols = ['Skills', 'education', 'Experience_Period','Designation']
resume_df['Resume'] = resume_df[cols].astype(str).apply(lambda row: '_'.join(row.values.astype(object)), axis=1)

In [ ]:
resume_df['Resume'].iloc[0].replace("'","").replace("[",'').replace("]",'')

In [ ]:
resume_df

In [ ]:
job_text = jobs_df[['Job Title', 'Job Experience Required', 'Key Skills']].apply(lambda x: ' '.join(x.astype(str)), axis=1)
vectorizer = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')

In [ ]:
def givejob(resume_text,vectorizer):
    resume_features = vectorizer.fit_transform([resume_text]) 
    print(resume_features.shape)
    job_features = vectorizer.transform(job_text) 
    print(job_features.shape)
    # Compute the cosine similarity between the resume and job data
    top_job_indices = cosine_similarity(resume_features, job_features)
    prd_arr = sorted(top_job_indices[0],reverse = True)[:5]
    top_job_indices=top_job_indices.argsort()[0][::-1][:5]
    jobs = jobs_df.iloc[top_job_indices]['Job Title'].tolist()
    return jobs,prd_arr

In [ ]:
jobs, prd_arr = givejob(resume_df['Resume'][1],vectorizer)

In [ ]:
prd_arr

In [ ]:
jobs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
 
courses = jobs
values = prd_arr

fig = plt.figure(figsize = (25, 15))
 
# creating the bar plot
plt.bar(courses, values, color =['lightgreen','cyan','lightblue','green','red'],width = 0.4,edgecolor='blue',)
plt.xlabel("recomendate job",fontsize=20)
plt.ylabel("similarity in percentage",fontsize=20)
plt.title("jobs recomendation",fontsize = 20)
plt.legend(loc='best', fontsize=25)
plt.show()